In [3]:
import numpy as np
from scipy.interpolate import UnivariateSpline 

In [3]:
def VQlinspace2(cycle_current, cycle_voltage, cycle_time): 
    # VQlinspace2 returns a linearly-spaced V vs Q curve
    # Inputs: cycle (i.e. batch(i).cycles(j))
    # Outputs: Qlin, Vlin = linearly spaced Qdis vs Vdis
    # VQlinspace2 uses time/current to generate discharge capacity due to
    # discrepancy for Qdischarge. This produces a "smoother" and more
    # physically meaningful discharge curve
    # Last modified October 9, 2017
    
    # 1. Get the indices of all currents ~ -4 C, i.e. discharge indices.
    # For all policies, we discharge at 4C (= -4.4A)
    start_index = np.argmax(np.abs(cycle_current + 4) < 0.05)
    end_index = np.argmin(cycle_voltage[start_index:-2]) + start_index

    # 2. Extract Q_dis (from t_dis) and V_dis:
    Q_dis_raw = -(cycle_time[start_index:end_index] - cycle_time[start_index]) / 60.0 * cycle_current[start_index:end_index] * 1.1
    V_dis_raw = cycle_voltage[start_index:end_index]

    # 3. Fit to function. Ensure data is nearly untransformed
    sorting_order = np.argsort(V_dis_raw, axis=0)
    VQfit = UnivariateSpline(
        np.take_along_axis(V_dis_raw, sorting_order, axis=0),
        np.take_along_axis(Q_dis_raw, sorting_order, axis=0)
    )

    # 5. Linearly interpolate
    Vlin = np.linspace(2.0, 3.5, 1000)
    Qlin = VQfit(Vlin)
    
    return Qlin,Vlin
    